In [1]:
!nvidia-smi

Thu Apr 20 20:08:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   42C    P8                5W /  N/A|   1424MiB /  8188MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!ls
!mkdir parm

'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
��Ŀ¼���ļ� parm �Ѿ����ڡ�


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import os

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [5]:
batch_size = 64
LR = 0.01
num_epochs = 80
dropout = 0.5


In [6]:
import sys 
print(sys.path)
sys.path.append(os.getcwd() + '\\..')
print(sys.path)

['e:\\Desktop\\study\\deep_learning\\BatchNormlization\\PythonExtension\\cifar10_verification', 'd:\\Anaconda\\envs\\unidl\\python39.zip', 'd:\\Anaconda\\envs\\unidl\\DLLs', 'd:\\Anaconda\\envs\\unidl\\lib', 'd:\\Anaconda\\envs\\unidl', '', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\mybatchnorm1d_cpp-0.0.0-py3.9-win-amd64.egg', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\mybatchnorm2d_cpp-0.0.0-py3.9-win-amd64.egg', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\win32', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\win32\\lib', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\Pythonwin']
['e:\\Desktop\\study\\deep_learning\\BatchNormlization\\PythonExtension\\cifar10_verification', 'd:\\Anaconda\\envs\\unidl\\python39.zip', 'd:\\Anaconda\\envs\\unidl\\DLLs', 'd:\\Anaconda\\envs\\unidl\\lib', 'd:\\Anaconda\\envs\\unidl', '', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages', 'd:\\Anaconda\\envs\\unidl\\lib\\site-packages\\mybatch

In [7]:
from MyBatchNormalization2D import MyBatchNormalization2d
from MyBatchNormalization1D import MyBatchNormalization1d
import torch.nn as nn
class Net_use_MyBn(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            #conv1 1/2
            nn.Conv2d(in_channels=3    , out_channels=64   , kernel_size=3,  padding = 1),
            MyBatchNormalization2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            #conv2 1/4 
            nn.Conv2d(in_channels=64   , out_channels=128, kernel_size=3,  padding = 1),
            MyBatchNormalization2d(num_features=128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #conv3 1/8 
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,  padding = 1),
            MyBatchNormalization2d(num_features=256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #conv4 1/16 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,  padding = 1),
            MyBatchNormalization2d(num_features=512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
             #conv5 1/32 
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,  padding = 1),
            MyBatchNormalization2d(num_features=1024),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),    
        )
    
        self.classifier = nn.Sequential(  # 定义自己的分类层
            nn.Dropout(),
            nn.Linear(in_features=1024, out_features=512), 
            MyBatchNormalization1d(512),
            nn.ReLU(True),
            nn.Linear(in_features=512, out_features=256),
            MyBatchNormalization1d(256),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(in_features=256, out_features=10),
        )
    

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

net3 = Net_use_MyBn()
net3.to(device)

Net_use_MyBn(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MyBatchNormalization2d()
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MyBatchNormalization2d()
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): MyBatchNormalization2d()
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): MyBatchNormalization2d()
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), pad

In [8]:
import torch.optim as optim
import torch.nn as nn
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def train_and_valid_MyBN(model, loss_function, optimizer, epochs):
    model.to(device)
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward(retain_graph=True)

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)
      

        avg_train_loss = train_loss / train_data_size
        avg_train_acc = train_acc / train_data_size

        avg_valid_loss = valid_loss / valid_data_size
        avg_valid_acc = valid_acc / valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print(
            "\t Training: Loss: {:.4f}, Accuracy: {:.4f}%, "
            "\n\t Validation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.3f}s".format(
                avg_train_loss, avg_train_acc * 100, avg_valid_loss, avg_valid_acc * 100,
                                epoch_end - epoch_start
            ))
        print("\t Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
        
        dir_path = os.path.join(os.getcwd(), ('parm/torchBN_batchsize_' + str(batch_size) + '_LR_' + str(LR) ) )
        if not os.path.exists(dir_path):
            os.mkdir(dir_path)
        model_path = dir_path + '/%02d' % (epoch + 1) + '.pt'
        torch.save(model, model_path)  # 保存模型
    
    print('Finished Training')

    return model, history

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net3.parameters(), lr= LR, momentum=0.9)


all_start = time.time()

norm_mean = [0.485, 0.456, 0.406]  # 均值
norm_std = [0.229, 0.224, 0.225]  # 方差
transform_train = transforms.Compose([transforms.ToTensor(),  # 将PILImage转换为张量
                                      # 将[0,1]归一化到[-1,1]
                                      transforms.Normalize(norm_mean, norm_std),
                                      transforms.RandomHorizontalFlip(),  # 随机水平镜像
                                      transforms.RandomErasing(scale=(0.04, 0.2), ratio=(0.5, 2)),  # 随机遮挡
                                      transforms.RandomCrop(32, padding=4)  # 随机中心裁剪
                                      ])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(norm_mean, norm_std)])
  # 

# 选择数据集:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
# 加载数据:
train_data = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
valid_data = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False)
cifar10_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

train_data_size = len(trainset)
valid_data_size = len(testset)
print(train_data_size, valid_data_size)

print("MyBN:")
print('Batch_Size = ' + str(batch_size) + ', Learning_Rate = ' + str(LR) + ', Train_Epochs = ' + str(num_epochs))
net3.to(device)

net3, history = train_and_valid_MyBN(model=net3, loss_function=loss_func, optimizer=optimizer, epochs=num_epochs )

Files already downloaded and verified
Files already downloaded and verified
50000 10000
MyBN:
Batch_Size = 64, Learning_Rate = 0.01, Train_Epochs = 80
Epoch: 1/80
	 Training: Loss: 1.5538, Accuracy: 42.6880%, 
	 Validation: Loss: 1.2490, Accuracy: 54.7700%, Time: 29.964s
	 Best Accuracy for validation : 0.5477 at epoch 001
Epoch: 2/80
	 Training: Loss: 1.1879, Accuracy: 57.6800%, 
	 Validation: Loss: 1.2940, Accuracy: 61.1900%, Time: 26.436s
	 Best Accuracy for validation : 0.6119 at epoch 002
Epoch: 3/80
	 Training: Loss: 1.0272, Accuracy: 63.6620%, 
	 Validation: Loss: 0.9564, Accuracy: 67.5400%, Time: 27.127s
	 Best Accuracy for validation : 0.6754 at epoch 003
Epoch: 4/80
	 Training: Loss: 0.9290, Accuracy: 67.7940%, 
	 Validation: Loss: 0.8449, Accuracy: 71.6200%, Time: 26.210s
	 Best Accuracy for validation : 0.7162 at epoch 004
Epoch: 5/80
	 Training: Loss: 0.8637, Accuracy: 70.0440%, 
	 Validation: Loss: 0.7229, Accuracy: 75.3200%, Time: 26.076s
	 Best Accuracy for validation :

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import os, sys
result_dir_path = os.path.join(os.getcwd(), ('result/torchBN_MyBN_batchsize_' + str(batch_size) + '_LR_' + str(LR) ) )
if not os.path.exists(result_dir_path):
    os.mkdir(result_dir_path)
history = np.array(history)
# Loss曲线
plt.figure(figsize=(10, 10))
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
# 设置坐标轴刻度
plt.xticks(np.arange(0, num_epochs + 1, step=10))
plt.yticks(np.arange(0, 2.05, 0.1))
plt.grid()  # 画出网格
plt.savefig(result_dir_path + '/loss_curve1.png')

# 精度曲线
plt.figure(figsize=(10, 10))
plt.plot(history[:, 2:4])
plt.legend(['Train Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
# 设置坐标轴刻度
plt.xticks(np.arange(0, num_epochs + 1, step=10))
plt.yticks(np.arange(0, 1.05, 0.05))
plt.grid()  # 画出网格
plt.savefig(result_dir_path + '/accuracy_curve1.png')